<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 6. Redes Neuronales Recurrentes</font></h1>

<h1><font color="#113D68" size=4>4. LSTM para Clasificación</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. LSTM para clasificación](#section1)
    * [1.1. Librerías](#section1.1)
    * [1.2. Dataset](#section1.2)
    * [1.3. Truncar las secuencias de entrada](#section1.3)
    * [1.4. Modelo de linea base](#section1.4)
    * [1.5. Resultados](#section1.5)
* [2. LSTM con Dropout](#section2)
* [3. LSTM con Dropout recurrente](#section3)
* [4. LSTM y CNN para clasificación](#section4)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

En este proyecto, vamos a desarrollar modelos LSTM para problemas de clasificación de secuencias:

* Desarrollar un modelo LSTM para un problema de clasificación de secuencias.
* Reducir el sobreajuste en sus modelos LSTM mediante Dropout.
* Combinar modelos LSTM con CNN.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. LSTM para clasificación</font>

Para esta práctica vamos a utilizar el problema de clasificación de opiniones de revisión de películas de IMDB.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Puede obtener más información sobre el dataset [IMBD](http://ai.stanford.edu/~amaas/data/sentiment/)

<a id="section1.1"></a>
# <font color="#004D7F" size=5>1.1. Funciones</font>

Comencemos importando las clases y funciones necesarias para este modelo.

In [3]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

<a id="section1.2"></a>
# <font color="#004D7F" size=5>1.2. Dataset</font>

Limitamos el conjunto de datos a las 5.000 palabras principales y dividimos en train (50%) y test (50%).

In [4]:
# load the dataset but only keep the top n words, zero the rest
max_palabras=5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_palabras)


17473536/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the nda

<a id="section1.3"></a>
# <font color="#004D7F" size=5>1.3. Truncar las secuencias de entrada</font>

Necesitamos truncar y rellenar las secuencias de entrada para que todas tengan la misma longitud para el modelado. 

In [5]:
# truncate and pad input sequences
max_long = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_long)
X_test = sequence.pad_sequences(X_test, maxlen=max_long)


<a id="section1.4"></a>
# <font color="#004D7F" size=5>1.4. Modelo de linea base</font>

Ahora podemos definir nuestro modelo:
1. La primera capa es la capa Embedding que utiliza 32 vectores para representar cada palabra. 
2. La siguiente capa es la capa LSTM con 100 unidades de memoria (neuronas inteligentes). 
3. Usamos una capa de salida densa con una sola neurona y una función de activación sigmoidea
4. Compilamos con `binary_crossentropy`, optimizador `adam` y la métrica Accuracy. 
5. Ajustamos a 3 épocas y un tamaño de batch=64.

In [6]:
# create the model
embedding_length = 32
model = Sequential()
model.add(Embedding(max_palabras, embedding_length , input_length = max_long))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()
model.fit(X_train, y_train, epochs=3, batch_size=64)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
391/391 [==============================] - 31s 27ms/step - loss: 0.6009 - accuracy: 0.6513
Epoch 2/3
391/391 [==============================] - 11s 27ms/step - loss: 0.3346 - accuracy: 0.8631
Epoch 3/3
391/391 [==============================] - 10s 27ms/step - loss: 0.2693 - accuracy: 0.8946


<a id="section1.5"></a>
# <font color="#004D7F" size=5>1.5. Resultados</font>

Estimamos el rendimiento del modelo en revisiones no vistas.

In [7]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 87.45%


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. LSTM con Dropout</font>

Podemos regular el problema de sobreajuste en RNN agregando nuevas capas Dropout entre las capas Embedding y LSTM y las capas de salida LSTM y Dense.

In [9]:
# create the model
embedding_length = 32
model = Sequential()
model.add(Embedding(max_palabras, embedding_length , input_length = max_long))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()
model.fit(X_train, y_train, epochs=10, batch_size=64)

#evaluate model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 500, 32)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
391/391 [==============================] - 12s 27ms/step - loss: 0.6111 - accuracy: 0.6540
Epoch 2/10
3

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. LSTM con Dropout recurrente</font>

Alternativamente, Dropout se puede aplicar a las conexiones de entrada y recurrentes de las unidades de memoria con el LSTM de manera precisa y por separado. Para ello, configuraremos el `dropout` de entrada y `recurrent_dropout` para configurar el Dropout recurrente. 

In [ ]:
# create the model
???

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section4"></a>
# <font color="#004D7F" size=6>4. LSTM y CNN para clasificación</font>

Podemos agregar fácilmente una CNN unidimensional y capas de agrupación máxima después de la capa Embedding que luego alimentan las características consolidadas al LSTM.

Utilizaremos:
1. En la convolución un conjunto más pequeño de 32 entidades con un filtro de 3. 
2. La capa de agrupación la longitud estándar de 2.

In [ ]:
# libraries
???

# create the model
??? 

# Final evaluation of the model
???

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>